In [1]:
import pickle
from neo4j import GraphDatabase
from dotenv import dotenv_values
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from cdlib import algorithms, viz, evaluation

from hp_nlp_graph.neo4j import add_metrics_to_neo4j

Note: to be able to use all crisp methods, you need to install some additional packages:  {'graph_tool', 'wurlitzer', 'bayanpy', 'infomap'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'pyclustering', 'ASLPAw'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'infomap'}


In [2]:
book_number = 7
interactions_by_chapter = pickle.load(
    open(f"data/processed/{book_number}/interactions_by_chapter.pkl", "rb")
)

In [3]:
neo4j_config = dotenv_values(".env")
driver = GraphDatabase.driver(
    neo4j_config["NEO4J_URL"],
    auth=(neo4j_config["NEO4J_USER"], neo4j_config["NEO4J_PASSWORD"]),
)

In [4]:
dfs = []
for chapter, distances in interactions_by_chapter.items():
    df = pd.DataFrame.from_dict(
        dict(distances), orient="index", columns=["weight"]
    ).reset_index(names=["characters"])
    df[["source", "target"]] = pd.DataFrame(df.characters.tolist(), index=df.index)
    df["chapter"] = chapter
    df["book"] = 1
    df = df[["source", "target", "weight", "chapter", "book"]]
    dfs.append(df)
df = (
    pd.concat(dfs)
    .groupby(["source", "target"])
    .weight.sum()
    .sort_values(ascending=False)
    .reset_index()
)

In [5]:
G = nx.from_pandas_edgelist(df, "source", "target", ["weight"], create_using=nx.Graph())

In [6]:
eigen_centrality = nx.eigenvector_centrality(G, weight="weight")
betweenness_centrality = nx.betweenness_centrality(G, weight="weight")
degree_centrality = nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
pagerank = nx.pagerank(G, weight="weight")
hub_centrality, authority_centrality = nx.hits(G)
degree = dict(nx.degree(G))
weighted_degree = dict(nx.degree(G, weight="weight"))

metrics = {
    "eigen_centrality": eigen_centrality,
    "betweenness_centrality": betweenness_centrality,
    "degree_centrality": degree_centrality,
    "closeness_centrality": closeness_centrality,
    "pagerank": pagerank,
    "hub": hub_centrality,
    "authority": authority_centrality,
    "degree": degree,
    "weighted_degree": weighted_degree,
}
metrics_df = pd.DataFrame.from_dict(metrics)
metrics_df.index.name = "name"
metrics_df

,eigen_centrality,betweenness_centrality,degree_centrality,closeness_centrality,pagerank,hub,authority,degree,weighted_degree
name,,,,,,,,,
Harry Potter,6.166853e-01,0.146360,0.753695,0.769704,0.207267,2.015598e-01,2.015598e-01,153,10576
Hermione Granger,5.480483e-01,0.069499,0.507389,0.629535,0.096541,1.791353e-01,1.791353e-01,103,5324
Ronald Weasley,5.105921e-01,0.140580,0.576355,0.663449,0.089669,1.668908e-01,1.668908e-01,117,4822
Tom Riddle,1.387361e-01,0.076915,0.384236,0.572803,0.030580,4.534760e-02,4.534760e-02,78,1511
Albus Dumbledore,1.173293e-01,0.101813,0.334975,0.559785,0.027978,3.835050e-02,3.835050e-02,68,1341
...,...,...,...,...,...,...,...,...,...
Bane,4.131142e-67,0.000000,0.009852,0.009852,0.004902,2.195239e-22,-6.004359e-18,2,2
Magorian,4.131142e-67,0.000000,0.009852,0.009852,0.004902,-1.135761e-21,-4.188847e-19,2,2
Ronan,4.131142e-67,0.000000,0.009852,0.009852,0.004902,-1.558565e-21,1.323598e-18,2,2


In [7]:
louvain_comms = algorithms.louvain(G, weight="weight")
louvain = pd.DataFrame.from_dict(
    dict(louvain_comms.to_node_community_map()),
    orient="index",
    columns=["louvain"],
)
louvain.index.name = "name"

In [8]:
leiden_comms = algorithms.leiden(G, weights="weight")
leiden = pd.DataFrame.from_dict(
    dict(leiden_comms.to_node_community_map()),
    orient="index",
    columns=["leiden"],
)
leiden.index.name = "name"

In [9]:
girvan_newman_comms = algorithms.girvan_newman(G, level=5)
girvan_newman = pd.DataFrame.from_dict(
    dict(girvan_newman_comms.to_node_community_map()),
    orient="index",
    columns=["girvan_newman"],
)
girvan_newman.index.name = "name"

In [10]:
spectral_comms = algorithms.spectral(G, kmax=8)
spectral = pd.DataFrame.from_dict(
    dict(spectral_comms.to_node_community_map()),
    orient="index",
    columns=["spectral"],
)
spectral.index.name = "name"

In [11]:
metrics_df = metrics_df.join([louvain, leiden, girvan_newman, spectral])

In [12]:
metrics_df.to_csv(f"data/processed/{book_number}/metrics.csv")

In [ ]:
add_metrics_to_neo4j(driver, metrics_df.reset_index().to_dict("records"))

In [13]:
for col in metrics_df.columns:
    nx.set_node_attributes(G, metrics_df[col].to_dict(), col)

In [14]:
nx.write_gexf(G, f"data/processed/{book_number}/graph.gexf")